<a href="https://colab.research.google.com/github/JuanCastellanos89/scrap/blob/main/S30_Ea_2_Optimizando_la_productividad_en_el_mundo_del_software_JuanCamiloCastellanosRodriguez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**S30 - Evidencia de aprendizaje 2. Optimizando la productividad en el mundo del software**

In [3]:
# Instalar librerias
!pip install requests beautifulsoup4
!pip install GitPython

In [4]:
from git import Repo, GitCommandError

repo_url = "https://github.com/JuanCastellanos89/scrap"
local_repo_path = "/content/scrap"

try:
    # Intentar clonar el repositorio
    print(f"Clonando el repositorio desde {repo_url}...")
    repo = Repo.clone_from(repo_url, local_repo_path)
    print("Repositorio clonado correctamente.")
except GitCommandError as e:
    print(f"Error al clonar el repositorio: {e}")
except Exception as e:
    print(f"Ha ocurrido un error inesperado al clonar el repositorio: {e}")

Clonando el repositorio desde https://github.com/JuanCastellanos89/scrap...
Repositorio clonado correctamente.


In [5]:
import requests
from bs4 import BeautifulSoup
import json
import time

# URL del artículo: tarjetas de video AMD en Amazon
url = "https://www.amazon.com/s?k=amd+graphics+card"

# Configuración de los encabezados
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.amazon.com/",
    "Accept-Encoding": "gzip, deflate, br"
}

# Función para obtener la descripción de cada producto
def get_product_description(product_url):
    response = requests.get(product_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Intenta encontrar la descripción utilizando diferentes selectores
        description_tag = soup.select_one("#productDescription p")
        if not description_tag:
            description_tag = soup.select_one("div#feature-bullets ul li span")
        if description_tag:
            return description_tag.get_text(strip=True)
        else:
            return "No description available"
    else:
        return "Error retrieving product description"

# Realiza la solicitud HTTP para la página principal
response = requests.get(url, headers=headers)

# Comprueba el estado de la respuesta
if response.status_code == 200:
    # Analiza el contenido con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extrae los datos de los productos (nombres, precios y enlaces)
    product_names = soup.select("h2 .a-link-normal.a-text-normal")
    product_prices = soup.select(".a-price-whole")

    # Almacena los datos en listas
    products = []
    for name, price in zip(product_names, product_prices):
        product_info = {
            "name": name.get_text(strip=True),
            "price": price.get_text(strip=True),
            "url": "https://www.amazon.com" + name['href'],  # Construye la URL completa
            "description": "Loading..."  # Se actualizará después
        }
        products.append(product_info)

    # Obtiene la descripción de cada producto visitando su URL
    for product in products:
        print(f"Obteniendo descripción para: {product['name']}")
        product['description'] = get_product_description(product['url'])
        # Espera entre solicitudes para no ser bloqueado por Amazon
        time.sleep(2)  # Espera de 2 segundos entre solicitudes

    # Imprime los resultados
    for product in products:
        print(f"Nombre: {product['name']}")
        print(f"Precio: ${product['price']}")
        print(f"URL: {product['url']}")
        print(f"Descripción: {product['description']}")
        print("-" * 40)

    # Guarda los resultados en un archivo .JSON
    with open('productos_amd_con_descripciones.json', 'w') as json_file:
        json.dump(products, json_file, indent=4)

    print("Los datos han sido guardados en 'productos_amd_con_descripciones.json'")

else:
    print("Error al realizar la solicitud:", response.status_code)

Obteniendo descripción para: Radeon RX 7600 XT Gaming OC 16G Graphics Card, 3X WINDFORCE Fans 16GB 128-bit GDDR6, GV-R76XTGAMING OC-16GD Video Card
Obteniendo descripción para: AMD Radeon™ RX 7900 XT Phantom Gaming 20GB OC Graphics Card 320 Bit 7680 x 4320 0dB Silent Cooling 3 x DisplayPort™ 2.1 GDDR6 1 x HDMI™ 2.1 20Gbps PCI Express 4.0
Obteniendo descripción para: Radeon RX 7800 XT Gaming OC 16G Graphics Card, 3X WINDFORCE Fans 16GB 256-bit GDDR6, GV-R78XTGAMING OC-16GD Video Card
Obteniendo descripción para: Speedster QICK319 Radeon RX 6750XT CORE Gaming Graphics Card with 12GB GDDR6 HDMI 3xDP, AMD RDNA 2 RX-675XYJFDP
Obteniendo descripción para: Fighter AMD Radeon RX 6750 XT 12GB GDDR6 Graphics Card
Obteniendo descripción para: 11319-07-20G Pulse AMD Radeon RX 6650 XT Gaming Graphics Card with 8GB GDDR6, AMD RDNA 2
Obteniendo descripción para: Gaming GeForce RTX 3060 12GB 15 Gbps GDRR6 192-Bit HDMI/DP PCIe 4 Torx Twin Fan Ampere OC Graphics Card
Obteniendo descripción para: TUF Gam